The linear MPC problem can be formulated as follows:
$$
\begin{aligned}
    \min_{\bar{x}, \bar{u}} \quad & \sum_{k=0}^{N-1} \frac{1}{2} x_k^\top Q x_k + \frac{1}{2} u_k^\top R u_k + \frac{1}{2} u_N^\top P u_N \\
    \text{s.t.} \quad & x_{k+1} = A x_k + B u_k, \;  k = 0, \dots, N-1, \\
    & \underline{c} \leq D x_k \leq  \bar{c}, \quad\quad\quad k = 1, \dots, N, \\
    & u_{\text{lb}} \leq u \leq u_{\text{ub}}, \quad\quad\;\; k = 0, \dots, N-1.
\end{aligned}
$$
Here: 
- $Q$, $R$, $P$: cost matrices. 
- $\bar{x} := (x_0, \dots, x_{N})$: state sequence. 
- $\bar{u} := (u_0, \dots, u_{N-1})$: control sequence. 
- $N$: prediction horizon. 
- $u_{\text{lb}}$ and $\underline{c}$: lower bounds.
- $u_{\text{ub}}$ and $\bar{c}$: upper bounds.

## Condensed Formulation

We write the predicted states in matrix form:

$$
\begin{bmatrix}
x_0 \\ x_1 \\ \vdots \\ x_N
\end{bmatrix} = T x_0 + S 
\begin{bmatrix}
u_0 \\ u_1 \\ \vdots \\ u_{N-1}
\end{bmatrix},
$$
where
$$
T := \begin{bmatrix}
I \\ A \\ A^2 \\ \vdots \\ A^N \\
\end{bmatrix}, \quad
S := \begin{bmatrix}
0 & 0 & 0 & \cdots & 0 \\
B & 0 & 0 & \cdots & 0 \\
AB & B & 0 & \cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & 0 \\
A^{N-1}B & A^{N-2}B & A^{N-3}B & \cdots & B
\end{bmatrix}.
$$

We define $\tilde{T}$ and $\tilde{S}$ as:
$$
\begin{bmatrix}
x_0 \\ x_1 \\ \vdots \\ x_{N-1}
\end{bmatrix} = \tilde{T} x_0 + \tilde{S} 
\begin{bmatrix}
u_0 \\ u_1 \\ \vdots \\ u_{N-1}
\end{bmatrix}.
$$

We want to rewrite constraints in matrix form: 
$\blue{G \bar{u} \leq g}$. Let us derive it step by step.

$$
\underbrace
{\begin{cases}
    D \underbrace{(A x_k + B u_k)}_{x_{k+1}} \leq \bar{c}, &k = 0, \dots, N-1, \\
   -D (A x_k + B u_k) \leq -\underline{c}, &k = 0, \dots, N-1, \\
    I u_k \leq u_{\text{ub}}, &k = 0, \dots, N-1, \\
   -I u_k \leq -u_{\text{lb}}, &k = 0, \dots, N-1.
\end{cases}}_{} \\ 
\tilde{D} x_k +  \tilde{E} u_k \leq \tilde{b},\; k = 0, \dots, N-1
$$

$\tilde{D}, \tilde{E}$ and $\tilde{c}$ are derived as:
$$
\tilde{D} = \begin{bmatrix}
    D A \\
    -D A \\
    0 \\
    0
\end{bmatrix},\;
\tilde{E} = \begin{bmatrix}
    D B \\
    -D B \\
    I \\
    -I 
\end{bmatrix},\;
\tilde{b} = \begin{bmatrix}
    \bar{c} \\
    -\underline{c}\\
    u_{\text{ub}} \\
    -u_{\text{lb}}
\end{bmatrix}. \;
$$

$$
\begin{aligned}
    \tilde{D} x_0 +  \tilde{E} u_0 &\leq \tilde{b} \\
    \tilde{D} x_1 +  \tilde{E} u_1 &\leq \tilde{b} \\ 
    &\vdots \\
    \tilde{D} x_{N-1} +  \tilde{E} u_{N-1} &\leq \tilde{b}
\end{aligned} \quad \implies \quad
\bar{D} \begin{bmatrix}
    x_0 \\ x_1 \\ \vdots \\ x_{N-1} 
\end{bmatrix} + \bar{E}
\begin{bmatrix}
    u_0 \\ u_1 \\ \vdots \\ u_{N-1} 
\end{bmatrix} \leq \bar{b}
$$


$$
\begin{aligned}
    \bar{D} \left( \tilde{T}x_0 + \tilde{S} \bar{u} \right) + \bar{E} \bar{u} \leq \bar{b} \\
    \underbrace{\left( \bar{D}\tilde{S} + \bar{E} \right)}_{G} \bar{u} \leq 
    \underbrace{\bar{b} - \bar{D} \tilde{T} x_0}_{g}
\end{aligned}
$$

### Robust implementation

The condensing procedure mentioned above is suitable for stable systems. However, for an unstable system, the power of $A$ diverges rapidly, causing some elements of $A$ to become extremely large. Consequently, the Hessian $H$ becomes ill-conditioned, leading to numerical issues. Here is an example.

In [28]:
import numpy as np
from utils import gen_prediction_matrices
np.set_printoptions(precision=3, suppress=True)

A = np.array([
    [2.6, -0.05, -0.5, 1.], 
    [1., 0., 0., 0.],
    [0., 1., 0., 0.],
    [0., 0., 0., 1.],
])
B = np.array([0., 0., 0., 1.]).reshape(-1, 1)
dim_x = A.shape[0]
dim_u = B.shape[1]
print(f"eigenvalues: {np.linalg.eigvals(A)}")
N = 25
T, S = gen_prediction_matrices(A, B, N)

Q = np.eye(dim_x)
R = np.eye(dim_u)
Q_bar = np.zeros(((dim_x * (N + 1), dim_x * (N + 1))))
Q_bar[-dim_x:, -dim_x:] = Q
Q_bar[:dim_x * N, :dim_x * N] = np.kron(np.eye(N), Q) 
R_bar = np.kron(np.eye(N), R)

H = S.T @ Q_bar @ S + R_bar
print(np.all(np.linalg.eigvals(H) > 0))
print(f"eigenvalues of H:\n {np.linalg.eigvals(H)}")

eigenvalues: [ 2.5  0.5 -0.4  1. ]
False
eigenvalues of H:
 [ 1.095e+19  0.000e+00  6.182e+02  7.253e+01  3.255e+01  1.625e+01
 -8.306e+00  8.445e+00  5.116e+00  3.531e+00  2.692e+00  6.058e-01
  2.211e+00  1.916e+00  1.729e+00  1.609e+00  1.530e+00  1.472e+00
  1.425e+00  1.388e+00  1.362e+00  1.344e+00  1.333e+00  1.328e+00
  1.324e+00]


In principle, the cost Hessian $H$ is positive definite if $R$ is positive definite. Unfortunately, in this example, one computed eigenvalue is negative due to numerical issues. Therefore, it is essential to precondition the cost Hessian. Here, we introduce a simple-but-efficient way. The core idea is to first stabilize the system and then introduce additional inputs for optimization. Based on this approach, we parameterize the input as follows:

$$
u_k = K x_k + v_k,
$$
where $K$ is a stabilizing feedback gain and $v_k \in \mathbb{R}^{n_u}$ is an auxiliary variable. We can use optimal feedback gain for the infinite LQR as $K$. Given this control law, the dynamics becomes
$$
x_{k+1} = A x_k + B (K x_k + v_k) = (A + BK) x_k + B v_k = A_K x_k + B v_k. 
$$

The new prediction matrices are given by
$$
T := \begin{bmatrix}
I \\ A_K \\ A_K^2 \\ \vdots \\ A_K^N \\
\end{bmatrix}, \quad
S := \begin{bmatrix}
0 & 0 & 0 & \cdots & 0 \\
B & 0 & 0 & \cdots & 0 \\
A_K B & B & 0 & \cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & 0 \\
A_K^{N-1}B & A_K^{N-2}B & A_K^{N-3}B & \cdots & B
\end{bmatrix}.
$$